In [2]:
import torch, numpy

In [3]:
import torch, math
from torch.autograd import Function
class MyMul(Function):
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x,y)
        return x * y

    @staticmethod
    def backward(ctx, grad_output):
        x,y = ctx.saved_tensors
        return grad_output * y,grad_output * x

class MyMax(Function):
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x,y)
        return torch.maximum(x,y)
    
    @staticmethod
    def backward(ctx, grad_output):
        x,y  = ctx.saved_tensors
        
        maskx = x > y
        masky = x < y
        maskequal = x == y 

        grad_x = torch.where(maskx, grad_output, torch.where(maskequal, grad_output * 0.5, torch.zeros_like(grad_output)))
        grad_y = torch.where(masky, grad_output, torch.where(maskequal, grad_output * 0.5, torch.zeros_like(grad_output)))
        
        return grad_x, grad_y 
